In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
flow = pd.read_excel('Flows_Clean.xlsx')
ret = pd.read_excel('Returns_Clean.xlsx')

flow.head()

,Dates,55976,41072,27748,75259,40970,62519,21792,54084,55511,...,63829,44986,11891,22752,27780,59379,75162,27705,54594,48725
0,19930401,-4.696503,-23.581608,0.904076,1.960112,0.687935,-1.791999,-1.465184,-0.124006,-1.681277,...,-3.762768,-1.733699,1.126364,-7.138398,3.073116,1.617669,2.448601,3.493056,0.902557,-0.535050
1,19930402,-6.975709,-23.331232,0.394204,23.691814,-1.895012,0.551048,-1.488078,-0.170873,-1.588607,...,-0.419981,-2.612104,0.019097,-7.138398,2.033127,-2.418096,-1.523983,-9.594576,-10.226385,-0.274176
2,19930405,-5.642141,-19.851716,-1.597132,8.839406,0.475161,8.010732,0.298018,0.281993,1.024464,...,-0.474352,1.625321,1.149879,-6.056788,2.309195,4.493038,0.854999,2.615968,0.817579,1.478560
3,19930406,-6.975709,23.276153,-1.129123,14.299749,1.426277,9.120669,-0.317257,-0.059609,5.883830,...,-1.294707,1.255646,0.803975,-2.927825,-5.331582,1.635919,-9.876419,-2.862576,-3.755393,1.674835
4,19930407,-6.975709,19.204137,-1.221991,-5.184144,-1.638974,0.808131,-0.463507,0.610452,-1.209499,...,0.000000,8.642735,1.060310,-2.069588,-1.625845,2.295149,0.570760,5.847429,-1.580178,1.814051


In [3]:
print(ret.shape)
ret.head()

(1958, 101)


,Dates,55976,41072,27748,75259,40970,62519,21792,54084,55511,...,63829,44986,11891,22752,27780,59379,75162,27705,54594,48725
0,19930401,-0.019685,0.019391,-0.007282,-0.003891,0.083333,0.027027,0.005348,0.020000,-0.010417,...,0.108108,-0.019048,-0.017857,-0.028269,0.000000,0.006173,-0.023669,0.000000,0.031915,0.014433
1,19930402,-0.020080,0.002717,-0.002445,0.058594,-0.038462,-0.026316,-0.007979,-0.019608,-0.021053,...,0.000000,-0.016181,-0.031818,-0.010909,0.000000,-0.020450,0.012121,-0.032609,0.020619,-0.010163
2,19930405,-0.024590,-0.006775,-0.002451,0.022140,-0.040000,0.027027,0.005362,0.000000,0.021505,...,-0.048780,-0.003289,0.032864,0.003676,-0.005102,0.018789,-0.005988,-0.011236,-0.010101,0.026694
3,19930406,-0.067227,0.009550,-0.007371,0.025271,0.041667,0.052632,0.005333,0.000000,0.000000,...,-0.025641,-0.023102,-0.009091,-0.007326,0.005128,0.002049,-0.024096,-0.011364,-0.010204,0.014000
4,19930407,0.000000,0.005405,0.000000,0.021127,0.040000,-0.050000,0.002653,0.000000,0.010526,...,-0.013158,0.016892,0.018349,0.000000,0.010204,0.014315,0.000000,0.011494,0.000000,0.005917


In [4]:
ticker = ret.columns.tolist()[1:]
len(ticker)

100

In [5]:
def add_lag(df, lag, feature_name):   
    for i in range(lag):
        column_name = feature_name + '_lag_' + str(i+1)
        df[column_name] = df.groupby('PERMNO')[feature_name].shift(i+1)
    return df

## flow preprocessing

In [6]:
flow_new = pd.DataFrame(columns=['Dates','PERMNO','flow'])
for i in ticker:
    flow_app = pd.DataFrame(columns=['Dates','PERMNO','flow'])
    flow_app['Dates'] = flow['Dates']
    flow_app['PERMNO'] = i
    flow_app['flow'] = flow[i]
    flow_new = pd.concat([flow_new,flow_app])

print(flow_new.shape)
flow_new

(195800, 3)


,Dates,PERMNO,flow
0,19930401,55976,-4.696503
1,19930402,55976,-6.975709
2,19930405,55976,-5.642141
3,19930406,55976,-6.975709
4,19930407,55976,-6.975709
...,...,...,...
1953,20001222,48725,5.934239
1954,20001226,48725,-1.281796
1955,20001227,48725,3.428868
1956,20001228,48725,-11.641011


### add lags

In [7]:
lag = 3
feature_name = 'flow'
flow_lag = add_lag(flow_new, lag, feature_name)

In [9]:
flow_lag[flow_lag['PERMNO']==ticker[20]]

,Dates,PERMNO,flow,flow_lag_1,flow_lag_2,flow_lag_3
0,19930401,62308,-0.751972,NaN,NaN,NaN
1,19930402,62308,5.111166,-0.751972,NaN,NaN
2,19930405,62308,-2.042631,5.111166,-0.751972,NaN
3,19930406,62308,-3.290295,-2.042631,5.111166,-0.751972
4,19930407,62308,-0.738485,-3.290295,-2.042631,5.111166
...,...,...,...,...,...,...
1953,20001222,62308,0.256148,3.590947,-13.449414,-6.618240
1954,20001226,62308,-0.503420,0.256148,3.590947,-13.449414
1955,20001227,62308,1.434621,-0.503420,0.256148,3.590947
1956,20001228,62308,1.821971,1.434621,-0.503420,0.256148


## return preprocessing

In [8]:
ret_new = pd.DataFrame(columns=['Dates','PERMNO','ret'])
for i in ticker:
    ret_app = pd.DataFrame(columns=['Dates','PERMNO','ret'])
    ret_app['Dates'] = ret['Dates']
    ret_app['PERMNO'] = i
    ret_app['ret'] = ret[i]
    ret_new = pd.concat([ret_new,ret_app])

print(ret_new.shape)
ret_new

(195800, 3)


,Dates,PERMNO,ret
0,19930401,55976,-0.019685
1,19930402,55976,-0.020080
2,19930405,55976,-0.024590
3,19930406,55976,-0.067227
4,19930407,55976,0.000000
...,...,...,...
1953,20001222,48725,0.026415
1954,20001226,48725,0.003676
1955,20001227,48725,0.028083
1956,20001228,48725,-0.040380


### add lags

In [9]:
lag = 3
feature_name = 'ret'
ret_lag = add_lag(ret_new, lag, feature_name)

# X_small

In [10]:
data = pd.merge(ret_lag,flow_lag,on=['Dates','PERMNO'])
data.head()

,Dates,PERMNO,ret,ret_lag_1,ret_lag_2,ret_lag_3,flow,flow_lag_1,flow_lag_2,flow_lag_3
0,19930401,55976,-0.019685,NaN,NaN,NaN,-4.696503,NaN,NaN,NaN
1,19930402,55976,-0.020080,-0.019685,NaN,NaN,-6.975709,-4.696503,NaN,NaN
2,19930405,55976,-0.024590,-0.020080,-0.019685,NaN,-5.642141,-6.975709,-4.696503,NaN
3,19930406,55976,-0.067227,-0.024590,-0.020080,-0.019685,-6.975709,-5.642141,-6.975709,-4.696503
4,19930407,55976,0.000000,-0.067227,-0.024590,-0.020080,-6.975709,-6.975709,-5.642141,-6.975709


In [11]:
data_dropna = data.dropna().reset_index(drop=True)
data_dropna

,Dates,PERMNO,ret,ret_lag_1,ret_lag_2,ret_lag_3,flow,flow_lag_1,flow_lag_2,flow_lag_3
0,19930406,55976,-0.067227,-0.024590,-0.020080,-0.019685,-6.975709,-5.642141,-6.975709,-4.696503
1,19930407,55976,0.000000,-0.067227,-0.024590,-0.020080,-6.975709,-6.975709,-5.642141,-6.975709
2,19930408,55976,0.040541,0.000000,-0.067227,-0.024590,-6.975709,-6.975709,-6.975709,-5.642141
3,19930412,55976,0.038961,0.040541,0.000000,-0.067227,-3.351631,-6.975709,-6.975709,-6.975709
4,19930413,55976,-0.033333,0.038961,0.040541,0.000000,-2.878666,-3.351631,-6.975709,-6.975709
...,...,...,...,...,...,...,...,...,...,...
195495,20001222,48725,0.026415,0.037859,-0.030380,0.066127,5.934239,5.569106,-3.791130,7.960677
195496,20001226,48725,0.003676,0.026415,0.037859,-0.030380,-1.281796,5.934239,5.569106,-3.791130
195497,20001227,48725,0.028083,0.003676,0.026415,0.037859,3.428868,-1.281796,5.934239,5.569106
195498,20001228,48725,-0.040380,0.028083,0.003676,0.026415,-11.641011,3.428868,-1.281796,5.934239


In [ ]:
data_dropna.to_excel('X_small.xlsx')

# X_full

In [12]:
X_small = data_dropna
dt = X_small[X_small.PERMNO==ticker[0]][['Dates','ret_lag_1','flow_lag_1']]
dt = pd.concat([dt]*100,axis=0).reset_index(drop=True)
dt.columns = ['Dates','ret_lag1_'+str(ticker[0]),'flow_lag1_'+str(ticker[0])]
#dt = pd.DataFrame(np.repeat(dt.values, 100, axis=0), columns=dt.columns)
dt

,Dates,ret_lag1_55976,flow_lag1_55976
0,19930406,-0.024590,-5.642141
1,19930407,-0.067227,-6.975709
2,19930408,0.000000,-6.975709
3,19930412,0.040541,-6.975709
4,19930413,0.038961,-3.351631
...,...,...,...
195495,20001222,0.057471,3.100924
195496,20001226,0.014493,3.519515
195497,20001227,-0.035714,0.278894
195498,20001228,0.038272,3.668618


In [13]:
X_full = X_small[['Dates','PERMNO','ret']]
X_small = data_dropna

start =time.process_time()

for i in ticker:
    dt = X_small[X_small.PERMNO==i][['ret_lag_1','ret_lag_2','flow_lag_1','flow_lag_2']]
    dt = pd.concat([dt]*100,axis=0).reset_index(drop=True)
    dt.columns = ['ret_lag1_'+str(i),'ret_lag2_'+str(i),'flow_lag1_'+str(i),'flow_lag2_'+str(i)]
    X_full = pd.concat([X_full,dt], axis=1)

end = time.process_time()
print('Running time: %s Seconds'%(end-start))

Running time: 44.109375 Seconds


In [14]:
X_full

,Dates,PERMNO,ret,ret_lag1_55976,ret_lag2_55976,flow_lag1_55976,flow_lag2_55976,ret_lag1_41072,ret_lag2_41072,flow_lag1_41072,...,flow_lag1_27705,flow_lag2_27705,ret_lag1_54594,ret_lag2_54594,flow_lag1_54594,flow_lag2_54594,ret_lag1_48725,ret_lag2_48725,flow_lag1_48725,flow_lag2_48725
0,19930406,55976,-0.067227,-0.024590,-0.020080,-5.642141,-6.975709,-0.006775,0.002717,-19.851716,...,2.615968,-9.594576,-0.010101,0.020619,0.817579,-10.226385,0.026694,-0.010163,1.478560,-0.274176
1,19930407,55976,0.000000,-0.067227,-0.024590,-6.975709,-5.642141,0.009550,-0.006775,23.276153,...,-2.862576,2.615968,-0.010204,-0.010101,-3.755393,0.817579,0.014000,0.026694,1.674835,1.478560
2,19930408,55976,0.040541,0.000000,-0.067227,-6.975709,-6.975709,0.005405,0.009550,19.204137,...,5.847429,-2.862576,0.000000,-0.010204,-1.580178,-3.755393,0.005917,0.014000,1.814051,1.674835
3,19930412,55976,0.038961,0.040541,0.000000,-6.975709,-6.975709,-0.005376,0.005405,0.560749,...,-4.991003,5.847429,0.000000,0.000000,-1.221645,-1.580178,0.001961,0.005917,0.205041,1.814051
4,19930413,55976,-0.033333,0.038961,0.040541,-3.351631,-6.975709,0.017568,-0.005376,-1.993177,...,-2.804367,-4.991003,0.020619,0.000000,-4.795620,-1.221645,0.019569,0.001961,0.819203,0.205041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195495,20001222,48725,0.026415,0.057471,0.020781,3.100924,2.088885,0.023358,-0.021429,-2.189652,...,-2.384491,-8.178128,0.103030,-0.098361,-7.881403,-8.396286,0.037859,-0.030380,5.569106,-3.791130
195496,20001226,48725,0.003676,0.014493,0.057471,3.519515,3.100924,0.025678,0.023358,-2.243636,...,24.041366,-2.384491,-0.010989,0.103030,-11.480184,-7.881403,0.026415,0.037859,5.934239,5.569106
195497,20001227,48725,0.028083,-0.035714,0.014493,0.278894,3.519515,0.034771,0.025678,0.645105,...,8.264004,24.041366,-0.027778,-0.010989,3.873294,-11.480184,0.003676,0.026415,-1.281796,5.934239
195498,20001228,48725,-0.040380,0.038272,-0.035714,3.668618,0.278894,0.001344,0.034771,-0.371168,...,-17.404165,8.264004,0.005714,-0.027778,4.045013,3.873294,0.028083,0.003676,3.428868,-1.281796


In [ ]:
X_full.to_csv('X_full.csv',mode='a', encoding='utf-8')